In [32]:
import pandas as pd 
from glob import glob
import os
import sys
import tkinter as tk
from tkinter import filedialog
import csv
import json
import openpyxl
import xlsxwriter

In [33]:
root=tk.Tk()
root.withdraw()
file1=filedialog.askopenfilename(title="Browse the required file...")

In [34]:
df=pd.read_csv(file1)
file_name='Raw_data_for_'+(os.path.splitext(file1)[0]).split('/')[-1]

In [35]:
df['event_data'].astype('str')
df['event_data']=df['event_data'].apply(json.loads)
df2=pd.json_normalize(df['event_data'])
df3=pd.concat([df.drop(['event_data'],axis=1),df2],axis=1)
del df,df2
df3["type"]=df3["type"].replace("apron_management.","",regex=True)
cols = []
count = 1
for column in df3.columns:
    if column == 'time':
        cols.append(f'time.{count}')
        count+=1
        continue
    cols.append(column)
df3.columns = cols
df3.drop('version', inplace=True, axis=1)

In [36]:
stage_changes=df3[(df3['process'] != "alarm_housekeeper") & (df3['type'] == "dgs_state_changes")& (df3['event_type'] == "SafedockStateChangeEvent")]
stage_changes['VDGS_state']=stage_changes.prev_safedock_state.str.cat(stage_changes.safedock_state,sep='_To_')
stage_changes['Flight_ID']=stage_changes['carrier'].str.cat(stage_changes['flight_nr'],sep=' ')
stage_cols=["id", "type", "time.2", "ifplid", "ac_type", "apron_id", "stand_id", "VDGS_state","event_classification", "Flight_ID"]
stage_changes=stage_changes[stage_cols]
stage_changes.rename(columns = {"id":"ID", "type":"Type", "time.2":"Time", "ifplid":"IFPLID", "ac_type":"Aircraft_Type", "apron_id":"Apron_ID", "stand_id":"Stand_ID", "VDGS_state":"VDGS_State","event_classification":"Event_Classification"},inplace=True)
stage_changes['Type']=stage_changes['Type'].replace(to_replace ="dgs_state_changes",value ="Stage Changes")

C:\Users\jamit\AppData\Local\Temp\ipykernel_12404\3917250090.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stage_changes['VDGS_state']=stage_changes.prev_safedock_state.str.cat(stage_changes.safedock_state,sep='_To_')
C:\Users\jamit\AppData\Local\Temp\ipykernel_12404\3917250090.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stage_changes['Flight_ID']=stage_changes['carrier'].str.cat(stage_changes['flight_nr'],sep=' ')
C:\Users\jamit\AppData\Local\Temp\ipykernel_12404\3917250090.py:6: SettingWithC

In [37]:
filter_col = [col for col in df3 if col.startswith('alarm.')]
alarms=df3[(df3['type'] == "alarms")& (df3['event_type'] == "AlarmOccurrencesEvent")]
alarm_cols=["id", "host", "process", "time.1", "type", "site", "time.2", "event_classification",'apron_id', "alarm", "event_src", "event_type", "location_info", "schedule_info"]
alarm_cols=alarm_cols+filter_col
alarms=alarms[alarm_cols]
alarms=alarms.explode('schedule_info')
alarms=alarms.explode('location_info')
alarms['alarm.alarm-description'].dropna(inplace=True)
alarm_location=pd.json_normalize(alarms['location_info'])
alarm_location=alarm_location.add_prefix('location_info.')
alarm_schedule=pd.json_normalize(alarms['schedule_info'])
alarm_schedule=alarm_schedule.add_prefix('schedule_info.')
alarms=pd.concat([alarms.reset_index(),alarm_location.reset_index(),alarm_schedule.reset_index()],axis=1)
alarms['Flight_ID']=alarms['schedule_info.carrier'].str.cat(alarms['schedule_info.flight_nr'],sep=' ')
alarms=alarms[['id','type', 'site', 'time.2','event_classification','alarm.alarm-id','location_info.apron_id','alarm.stand-id','alarm.alarm-code', 'alarm.alarm-description','schedule_info.ac_type','Flight_ID']]
alarms.rename(columns = {'id':'ID','type':'Type','time.2':'Time','alarm.alarm-id':'Alarm_ID','location_info.apron_id':'Apron_ID','alarm.stand-id':'Stand_ID','alarm.alarm-code':'Alarm_Code', 'alarm.alarm-description':'Alarm_Description','schedule_info.ac_type':'Aircraft_Type'}, inplace = True)

In [38]:
error_mapping={'Minor display error - faulty LEDs detected':'Technical','IDFailed - IdLostTrackCloseToStop':'Operational','ApronBlocked':'Operational','IDFailed - NoseHeightFailed':'Operational','GateBlocked':'Operational','Major display error - faulty LED board detected':'Technical','IDFailed - VerAnalysisFailedTooClose':'Operational','AutoCalibrationError':'Technical','BridgeNotIn':'Operational','SBUStop - AzimuthStop':'Technical','SBUStop - ScanFailed':'Technical','IDFailed - EngineCheckFailed':'Operational','ConfigurationError - SystemIntegrity':'Technical','SBUStop - LdmDataRate':'Technical','ViewBlocked':'Operational','IDFailed - ProfileFailed':'Operational'}
stage_change_alarms=pd.concat([stage_changes,alarms],axis=0,ignore_index=True)
stage_change_alarms['Error_Type'] = stage_change_alarms['Alarm_Description'].map(error_mapping)
stage_change_alarms['Time']=pd.to_datetime(stage_change_alarms['Time'])

In [39]:
check=df3[(df3['type'] == "alarms")& (df3['event_type'] == "AlarmOccurrencesEvent")]
check=check['alarm.alarm-description'].value_counts().reset_index()

In [40]:
###SAM_sequencing###
sam_sequencing=df3[(df3['type'] == "flight_updates")&(df3['event_type']!='FlightTerminatedEvent')]
sam_sequencing['carrier_nr']=sam_sequencing['carrier'].str.cat(sam_sequencing['flight_nr'],sep=' ')
sam_sequencing['SAM_sequencing_state']=sam_sequencing['flight_plan_diff.fields.sequencing-state.from'].str.cat(sam_sequencing['flight_plan_diff.fields.sequencing-state.to'],sep='_To_')
# sam_filter_col = [col for col in sam_sequencing if col.startswith('flight_plan')]
sam_col=['id','flight_plan.timestamps.SEQT','apron_id','flight_plan.fields.stand','flight_plan.fields.flight-plan-type','carrier_nr','flight_plan.fields.update-counter', 'ac_type','flight_plan.fields.actual-dgs-aircraft-type','flight_plan.fields.tow-in','SAM_sequencing_state','flight_plan_diff.fields.sequencing-state.to','flight_plan.timestamps.EIBT','flight_plan.timestamps.DETECTED_AIBT','flight_plan.timestamps.EOBT','flight_plan.timestamps.DETECTED_AOBT']
# sam_col=sam_col+sam_filter_col
sam_sequencing=sam_sequencing[sam_col]
sam_sequencing['flight_plan.timestamps.SEQT']=pd.to_datetime(sam_sequencing['flight_plan.timestamps.SEQT'])
sam_sequencing.rename(columns={'flight_plan.timestamps.SEQT':'SEQT','flight_plan.fields.stand':'Stand','flight_plan.fields.flight-plan-type':'flight-plan-type','flight_plan.fields.update-counter':'update-counter','flight_plan.fields.actual-dgs-aircraft-type':'actual-dgs-aircraft-type','flight_plan.fields.tow-in':'tow-in','flight_plan_diff.fields.sequencing-state.to':'ENd_Sequence','flight_plan.timestamps.EIBT':'EIBT','flight_plan.timestamps.DETECTED_AIBT':'DETECTED_AIBT','flight_plan.timestamps.EOBT':'EOBT','flight_plan.timestamps.DETECTED_AOBT':'DETECTED_AOBT'},inplace=True)

C:\Users\jamit\AppData\Local\Temp\ipykernel_12404\449847485.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sam_sequencing['carrier_nr']=sam_sequencing['carrier'].str.cat(sam_sequencing['flight_nr'],sep=' ')
C:\Users\jamit\AppData\Local\Temp\ipykernel_12404\449847485.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sam_sequencing['SAM_sequencing_state']=sam_sequencing['flight_plan_diff.fields.sequencing-state.from'].str.cat(sam_sequencing['flight_plan_diff.fields.sequencing-state.to'],sep='_To_')
C:\

In [41]:
with pd.ExcelWriter("C:/Users/jamit\Desktop/"+file_name+".xlsx") as writer:
    stage_change_alarms.to_excel(writer, sheet_name="VDGS_stagechanges_alarms", index=False)
    check.to_excel(writer, sheet_name="Errors", index=False)
    sam_sequencing.to_excel(writer, sheet_name="Sam_Sequencing", index=False)



PermissionError: [Errno 13] Permission denied: 'C:/Users/jamit\\Desktop/Raw_data_for_2022-08-16.xlsx'